In [5]:
#encoding 
#take points and build a polygon
# two rasters 
# three text with codinates 
#1 and 3 steps concurrently runs 
#escape characters - clean the data from the files
# part 2 zonal geometry - how in python 
#IDE want more genaric 
# list of coordinates 
#create dataframe single column that has the geometry 
#add then the names 
#use a dictionary 
#open file set up polggon create a dicition 
#geodata frame is created from the dictionary 
#number of coordinate and name of dictionary 
#put t
#glob.glob('./lab2_data/data/districts/*.txt')
#glob.glob('../lab2_data/data/*/*')
#data = open(lab_data, 'r')
#print(data.read())
#for name in lab_data: 
   # print(name) 
#glob.glob('../lab2_data/data/agriculture/*.tif')

In [1]:
import geopandas as gpd
import pandas as pd
import glob
import rasterio
from shapely.geometry import Point, LineString, Polygon

In [2]:
f_dist = glob.glob(r'C:\Users\Aubre_M\Documents\Fall 2020\GIS Programming_ Automation\geog_4092\Week 2\lab2\lab2_data\data\districts\*.txt')
df_data = {'district':[], 'num_coords':[],'geometry':[]}
           
for i in f_dist:
    with open(i) as f:
        lines = f.readlines()
        coord_list = [] 
        for l in lines[1:]:
            coord = [float(coord) for coord in l.split('\t')]
            coord_list.append(coord)
                  
        district_area = Polygon(coord_list)
        district_name = i[-6:-4]
        coord_lenght = len(coord_list)
        df_data['district'].append(district_name)
        df_data['num_coords'].append(coord_lenght)
        df_data['geometry'].append(district_area)
    
df_data   
#print(df_data)
crs = "EPSG:4326"
dis_gdf = gpd.GeoDataFrame(df_data, geometry = 'geometry', crs = crs)       
#type(coord_gdf)
dis_gdf

,district,num_coords,geometry
0,01,1847,"POLYGON ((39.07757 -6.61146, 39.07505 -6.61193..."
1,05,566,"POLYGON ((39.04198 -5.36686, 39.04034 -5.36935..."
2,06,1223,"POLYGON ((38.72445 -5.50916, 38.72850 -5.51931..."


In [8]:
from rasterstats import zonal_stats
f_gemo = glob.glob(r'C:\Users\Aubre_M\Documents\Fall 2020\GIS Programming_ Automation\geog_4092\Week 2\lab2\lab2_data\data\agriculture\*.tif')
f_dist = glob.glob(r'C:\Users\Aubre_M\Documents\Fall 2020\GIS Programming_ Automation\geog_4092\Week 2\lab2\lab2_data\data\districts\*.txt')
df_rdata = {'Year':[],'Ag_percent':[], 'district':[]}


for i in f_gemo:
    # createds a dictionary with the district names for each of the raster files
    for j in f_dist:
            district_name = j[-6:-4]
            df_rdata['district'].append(district_name)
    # uses the geodata frame from part one to create a reference for the raster and the area of intrest  builds a geojson 
    ag_area = zonal_stats(dis_gdf, i, geojsonout = True)  
        
        # takes the average of the rasters cells within the geojson that are the same as the output and then takes the it an multiplies by 100
        # rather than have to take the summ of the entire raster with the same zone as the districts the output then dividing that by the summ of the cells in the districts to find the perentage which in 
        # count and dviding the sum by the count which is the same as taking the avergage or mean. 
        #this case is the same 
    for p in (ag_area):
        perc = (p['mean'])*100
        df_rdata['Ag_percent'].append(perc)
        ag_year = i[-13:-9]
        df_rdata['Year'].append(ag_year)
        
df_rdata
per_df = pd.DataFrame(df_rdata)
per_df


,Year,Ag_percent,district
0,2004,40.002673,01
1,2004,35.722357,05
2,2004,45.835598,06
3,2009,55.108631,01
4,2009,34.700027,05
5,2009,45.322060,06


In [44]:
Final_df = dis_gdf.merge(per_df,on ='district')
final_df = gpd.GeoDataFrame(Final_df)
Final_df

print ('In District '+ str(per_df.iloc[0,2]) + ' had '
       + str(per_df.iloc[0,1]) + ' % of agircultural land in '+ str(per_df.iloc[0,0]) + '\n'+
      'In District '+ str(per_df.iloc[3,2]) + ' had '
       + str(per_df.iloc[3,1]) + '% of agircultural land in '+ str(per_df.iloc[3,0]) + '\n'+ '\n'+
       'In District '+ str(per_df.iloc[1,2]) + ' had '
       + str(per_df.iloc[1,1]) + ' % of agircultural land in '+ str(per_df.iloc[1,0]) + '\n'+
       'In District '+ str(per_df.iloc[4,2]) + ' had '
       + str(per_df.iloc[4,1]) + ' % of agircultural land in '+ str(per_df.iloc[4,0]) + '\n'+ '\n'+
       
       'In District '+ str(per_df.iloc[2,2]) + ' had '
       + str(per_df.iloc[2,1]) + ' % of agircultural land in '+ str(per_df.iloc[2,0]) + '\n'+
       'In District '+ str(per_df.iloc[5,2]) + ' had '
       + str(per_df.iloc[5,1]) + ' % of agircultural land in '+ str(per_df.iloc[5,0]) + '\n')

In District 01 had 40.002672635552734 % of agircultural land in 2004
In District 01 had 55.108631499237184% of agircultural land in 2009

In District 05 had 35.722356739305894 % of agircultural land in 2004
In District 05 had 34.700026903416735 % of agircultural land in 2009

In District 06 had 45.835598376108905 % of agircultural land in 2004
In District 06 had 45.3220601672469 % of agircultural land in 2009



In [70]:
import rasterio.features
import rasterio.warp
import numpy 
with rasterio.open(r'C:\Users\Aubre_M\Documents\Fall 2020\GIS Programming_ Automation\geog_4092\Week 2\lab2\lab2_data\data\agriculture\GLOBCOVER_2004_lab2.tif') as dataset:

    # Read the dataset's valid data mask as a ndarray.
    mask = dataset.dataset_mask()

    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(
            mask, transform=dataset.transform):

        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326).
        geom = rasterio.warp.transform_geom(
            dataset.crs, "EPSG:4326", geom, precision=6)

        # Print GeoJSON shapes to stdout.
        print(geom)

{'type': 'Polygon', 'coordinates': [[[37.711688, -4.961577], [37.711688, -6.781216], [39.111837, -6.781216], [39.111837, -4.961577], [37.711688, -4.961577]]]}
